# [Day 14: Regolith Reservoir](https://adventofcode.com/2022/day/14)

The distress signal leads you to a giant waterfall! Actually, hang on - the signal seems like it's coming from the waterfall itself, and that doesn't make any sense. However, you do notice a little path that leads behind the waterfall.

Correction: the distress signal leads you behind a giant waterfall! There seems to be a large cave system here, and the signal definitely leads further inside.

As you begin to make your way deeper underground, you feel the ground rumble for a moment. Sand begins pouring into the cave! If you don't quickly figure out where the sand is going, you could quickly become trapped!

Fortunately, your familiarity with analyzing the path of falling material will come in handy here. You scan a two-dimensional vertical slice of the cave above you (your puzzle input) and discover that it is mostly air with structures made of rock.

Your scan traces the path of each solid rock structure and reports the x,y coordinates that form the shape of the path, where x represents distance to the right and y represents distance down. Each path appears as a single line of text in your scan. After the first point of each path, each point indicates the end of a straight horizontal or vertical line to be drawn from the previous point. For example:

```
498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
```

This scan means that there are two paths of rock; the first path consists of two straight lines, and the second path consists of three straight lines. (Specifically, the first path consists of a line of rock from 498,4 through 498,6 and another line of rock from 498,6 through 496,6.)

The sand is pouring into the cave from point 500,0.

Drawing rock as #, air as ., and the source of the sand as +, this becomes:

```
  4     5  5
  9     0  0
  4     0  3
0 ......+...
1 ..........
2 ..........
3 ..........
4 ....#...##
5 ....#...#.
6 ..###...#.
7 ........#.
8 ........#.
9 #########.
```

Sand is produced one unit at a time, and the next unit of sand is not produced until the previous unit of sand comes to rest. A unit of sand is large enough to fill one tile of air in your scan.

A unit of sand always falls down one step if possible. If the tile immediately below is blocked (by rock or sand), the unit of sand attempts to instead move diagonally one step down and to the left. If that tile is blocked, the unit of sand attempts to instead move diagonally one step down and to the right. Sand keeps moving as long as it is able to do so, at each step trying to move down, then down-left, then down-right. If all three possible destinations are blocked, the unit of sand comes to rest and no longer moves, at which point the next unit of sand is created back at the source.

So, drawing sand that has come to rest as o, the first unit of sand simply falls straight down and then stops:

```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
......o.#.
#########.
```

The second unit of sand then falls straight down, lands on the first one, and then comes to rest to its left:

```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
.....oo.#.
#########.
```

After a total of five units of sand have come to rest, they form this pattern:

```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
......o.#.
....oooo#.
#########.
```

After a total of 22 units of sand:

```
......+...
..........
......o...
.....ooo..
....#ooo##
....#ooo#.
..###ooo#.
....oooo#.
...ooooo#.
#########.
```

Finally, only two more units of sand can possibly come to rest:

```
......+...
..........
......o...
.....ooo..
....#ooo##
...o#ooo#.
..###ooo#.
....oooo#.
.o.ooooo#.
#########.
```

Once all 24 units of sand shown above have come to rest, all further sand flows out the bottom, falling into the endless void. Just for fun, the path any new sand takes before falling forever is shown here with ~:

```
.......+...
.......~...
......~o...
.....~ooo..
....~#ooo##
...~o#ooo#.
..~###ooo#.
..~..oooo#.
.~o.ooooo#.
~#########.
~..........
~..........
~..........
```

Using your scan, simulate the falling sand. How many units of sand come to rest before sand starts flowing into the abyss below?

In [9]:
import dotenv
import os
import requests

dotenv.load_dotenv()

input = requests.get(
  'https://adventofcode.com/2022/day/14/input',
  cookies={'session': os.getenv('ADVENT_OF_CODE_SESSION') }
)

In [23]:
text="""\
498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
"""

text = input.text

In [11]:
import re

coord_re = re.compile(r'(\d+),(\d+)')

def find_dims(text):
  coords = []
  offset = 0
  while True:
    m = coord_re.search(text[offset:])
    if not m:
      break
    coords.append((int(m.group(1)), int(m.group(2))))
    offset += m.end()

  min_x = min(coords, key=lambda c: c[0])[0]
  max_x = max(coords, key=lambda c: c[0])[0]
  max_y = max(coords, key=lambda c: c[1])[1]

  return min_x, max_x, max_y

In [12]:
def make_grid(text, min_x, max_x, max_y):
  grid = [['.' for _ in range(max_x + 1 - min_x)] for _ in range(max_y + 1)]

  for line in text.splitlines():
    offset = 0
    last = None
    while True:
      m = coord_re.search(line[offset:])
      if not m:
        break
      offset += m.end()
      x = int(m.group(1)) - min_x
      y = int(m.group(2))
      grid[y][x] = '#'
      if last:
        if last[0] == x:
          for yy in range(min(last[1], y), max(last[1], y) + 1):
            grid[yy][x] = '#'
        else:
          for xx in range(min(last[0], x), max(last[0], x) + 1):
            grid[y][xx] = '#'
      last = (x, y)

  return grid

In [21]:
def produce_sand(grid, min_x, verbose=False):
  x = 500 - min_x
  y = 0
  while True:
    if (verbose):
      print(f'x: {x}, y: {y}')
    if grid[y][x] != '.':
      raise Exception("start")
    elif y == len(grid) - 1:
      # can't go down
      raise Exception("down")
    elif grid[y + 1][x] == '.':
      y += 1
    elif x == 0:
      # can't go left
      raise Exception("left")
    elif grid[y + 1][x - 1] == '.':
      x -= 1 
      y += 1
    elif x == len(grid[0]) - 1:
      # can't go right
      raise Exception("right")
    elif grid[y + 1][x + 1] == '.':
      x += 1
      y += 1
    else:
      grid[y][x] = 'o'
      break


In [16]:
def print_grid(grid):
  print('\n'.join([''.join(line) for line in grid]))

def produce_sands(grid, min_x, end="down"):
  count = 0
  while True:
    try:
      produce_sand(grid, min_x)
      count += 1
      # if count % 100 == 0:
      #   print_grid(grid)
    except Exception as exc:
      if exc.args[0] == end:
        return count
      else:
        raise exc

In [17]:
# no record destructuring in Python?
min_x, max_x, max_y = find_dims(text)

grid = make_grid(text, min_x, max_x, max_y)

produce_sands(grid, min_x)


961

You realize you misread the scan. There isn't an endless void at the bottom of the scan - there's floor, and you're standing on it!

You don't have time to scan the floor, so assume the floor is an infinite horizontal line with a y coordinate equal to two plus the highest y coordinate of any point in your scan.

In the example above, the highest y coordinate of any point is 9, and so the floor is at y=11. (This is as if your scan contained one extra rock path like -infinity,11 -> infinity,11.) With the added floor, the example above now looks like this:

```
        ...........+........
        ....................
        ....................
        ....................
        .........#...##.....
        .........#...#......
        .......###...#......
        .............#......
        .............#......
        .....#########......
        ....................
<-- etc #################### etc -->
```

To find somewhere safe to stand, you'll need to simulate falling sand until a unit of sand comes to rest at 500,0, blocking the source entirely and stopping the flow of sand into the cave. In the example above, the situation finally looks like this after 93 units of sand come to rest:

```
............o............
...........ooo...........
..........ooooo..........
.........ooooooo.........
........oo#ooo##o........
.......ooo#ooo#ooo.......
......oo###ooo#oooo......
.....oooo.oooo#ooooo.....
....oooooooooo#oooooo....
...ooo#########ooooooo...
..ooooo.......ooooooooo..
#########################
```

Using your scan, simulate the falling sand until the source of the sand becomes blocked. How many units of sand come to rest?

In [24]:
min_x, max_x, max_y = find_dims(text)

grid = make_grid(text, 0, max_x + 500, max_y + 2)

for x in range(0, max_x + 500):
  grid[max_y + 2][x] = '#'

produce_sands(grid, 0, "start")


26375